In [ ]:
# Ensure required packages are installed
# pip install pandas numpy

import sys
import hashlib
import urllib.request, urllib.error, urllib.parse
import hmac
import base64
import urllib.parse
import time
import pandas as pd
import numpy as np
from google_maps_geocoder.geocoder import GoogleGeocoder
import os
import requests

# Load the dataset
data = pd.read_csv(r"C:\Users\Miles.Vandenburg\Documents\Python_Projects\Google_Maps_Geocoder\Examples\example_address.csv")

geocoder = GoogleGeocoder()
# Clean and prepare the data
destinations, needs_geocoding = geocoder.cleanup_pd(data)

# # Ensure the CSV contains a column named 'Address'
# if 'Address' not in data.columns:
#     raise ValueError("Input CSV must contain an 'Address' column.")

# Create an empty DataFrame to store signed URLs
MasterDF = pd.DataFrame([], columns=['Signed_URL'], dtype='str')
pd.set_option("display.max_colwidth", 254)

# Google Maps API Private Key (Replace with your actual private key)
privateKey = "Insert Key here"

# Base URL for Google Maps Geocoding API
base_url = "http://maps.googleapis.com/maps/api/geocode/json"
client = "gme-cushmanwakefield"
channel = "GBC"

for address in destinations['ADDRESS_FULL']:
    # Construct the URL using the address
    query = urllib.parse.urlencode({"address": address, "client": client, "channel": channel})
    url = f"{base_url}?{query}"

    # Record the start time for rate limiting
    startTime = time.time()

    # Sign and assemble the URL
    parsed_url = urllib.parse.urlparse(url)
    urlToSign = parsed_url.path + "?" + parsed_url.query

    # Decode the private key and generate the signature
    decoded_key = base64.urlsafe_b64decode(privateKey)
    signature = hmac.new(decoded_key, urlToSign.encode(), hashlib.sha1)
    encodedSignature = base64.urlsafe_b64encode(signature.digest()).decode("utf-8")

    # Construct the signed URL
    originalUrl = parsed_url.scheme + "://" + parsed_url.netloc + parsed_url.path + "?" + parsed_url.query
    AssembledSignedURL = f"{originalUrl}&signature={encodedSignature}"

    # Append the signed URL to the DataFrame
    MasterDF = pd.concat([MasterDF, pd.DataFrame([AssembledSignedURL], columns=['Signed_URL'])])

    # Ensure query takes at least 0.1 seconds to respect Google rate limits
    duration = time.time() - startTime
    if duration < 0.11:
        time.sleep(0.11 - duration)

# Save the output to a CSV file (replace 'output.csv' with your desired output path)
MasterDF.to_csv('output.csv', index=False)

print("Signed URLs have been saved to output.csv")

# Ping google for the reuslts:
results_dest = []
for i in MasterDF['Signed_URL']:
    results = requests.get(i)
    # Results will be in JSON format - convert to dict using requests functionality
    results = results.json()

    # if there's no results or an error, return empty results.
    if len(results['results']) == 0:
        output = {
            "formatted_address" : None,
            "latitude": None,
            "longitude": None,
            "accuracy": None,
            "google_place_id": None,
            "type": None,
            "postcode": None
        }
    else:
        answer = results['results'][0]
        output = {
            "formatted_address" : answer.get('formatted_address'),
            "latitude": answer.get('geometry').get('location').get('lat'),
            "longitude": answer.get('geometry').get('location').get('lng'),
            "accuracy": answer.get('geometry').get('location_type'),
            "google_place_id": answer.get("place_id"),
            "type": ",".join(answer.get('types')),
            "postcode": ",".join([x['long_name'] for x in answer.get('address_components')
                                if 'postal_code' in x.get('types')])
        }

    # Append some other details:
    # output['input_string'] = address
    output['number_of_results'] = len(results['results'])
    output['status'] = results.get('status')
    results_dest.append(output)

google_geocode_resultsDF_destinations = pd.DataFrame(results_dest)

    # if return_full_response is True:
    #     output['response'] = results
    # return output


Signed URLs have been saved to output.csv
